## ML Pipelines with ZenML  

**Key Concepts:** *ML Pipelines, Steps*  

- In this notebook, techniques are shown on how to easily convert existing ML code into ML pipelines using ZenML.  
- Models will be built with [sklearn](https://scikit-learn.org).  
- ZenML & sklearn integration installed. Use the following command, which will also restart the kernel of your notebook:  

```bash
pip install zenml[server]
